In [ ]:
#@title Click Me { form-width: "10%" }
 
%%html
<!--    
,_._._._._._._._._|__________________________________________________________,
|_|_|_|_|_|_|_|_|_|_________________________________________________________/
        
 
Folder With All Saved Data
https://drive.google.com/open?id=1ak2PrL1t3pQCo8VYzHXm9zuOM0Og9cjC 
 
-->
 
<marquee style='width: 75%; color: #000000;'><h3>Enjoy :)</h3></marquee>

In [ ]:
#@title Geting The Dataset{ form-width: "10%" }

# Geting The Training Dataset
!gdown --id "1fu93ErsAOzDf9kK2ws4I2ILcMYr2SZbF"
# If You Wander What Is This Magic Numbers check the link 
# https://drive.google.com/file/d/1fu93ErsAOzDf9kK2ws4I2ILcMYr2SZbF/view
get_ipython().system_raw("unrar x Training_set")

# Geting The Testing Dataset
!gdown --id "1tfcRgZrMp8cVJeH-YY2YO0RJEe9dKMVy"
get_ipython().system_raw("unrar x Testing_set")

In [ ]:
#@title Libraries { form-width: "3%" }
 
# Helpers 
from sklearn.model_selection import train_test_split
from termcolor import colored, cprint
from numba import cuda
from tqdm import tqdm
import numpy as np
import csv
import cv2
import os
import re
 
# Keras
from keras.layers import Dense, Activation, Dropout, Bidirectional
from keras.applications.vgg16 import VGG16, preprocess_input
# from keras.applications import inception_v3
# from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.optimizers import SGD
from keras.utils import np_utils

In [ ]:
#@title Hyperparameter { form-width: "3%" }

## Pathes
# Pathes Of Training
TRAINING_PATH_FOLDER = '/content/Training'
TRAINING_NPY = '/content/drive/My Drive/Video Action Recognition/train.npy'
TRAINING_LABEL_NPY = '/content/drive/My Drive/Video Action Recognition/train_label.npy'

# Paths Of Testing
TESTING_PATH_FOLDER = '/content/Testing_set'
TESTING_NPY = '/content/drive/My Drive/Video Action Recognition/test.npy'
TESTING_LABEL_NPY = '/content/drive/My Drive/Video Action Recognition/test_label.npy'

# Path Of Hidden
HIDDEN_NPY = '/content/drive/My Drive/Video Action Recognition/hidden.npy'

# Path Of Weights
WEIGHTS_PATH = '/content/drive/My Drive/Video Action Recognition/weights.h5'

# Path Of CSV
CSV_PATH = '/content/drive/My Drive/Video Action Recognition/submit.csv'
CSV_SORTED_PATH = '/content/drive/My Drive/Video Action Recognition/sorted.csv'

## Flags
# True For Training False For Testing
# MASTER_FLAG = True
MASTER_FLAG = False
# True For Loading False For Saving
LOAD_SAVE = False
# LOAD_SAVE = True

# Labels In Video Names
LABELS = ["diving", "jumping", "shooting", "tennis", "walk"]

In [ ]:
#@title Didn't know there is Folders Preprocessing { form-width: "3%" }
'''
    So To Understand What We Did Here But Before That Let Me Show You The Structre
    Of Training & Testing Floders
    

├── Testing_set
│   ├── test_image (1).jpg
│   │ 
│   └──test_image (10).mpg.jpg
│
├── Training
│   ├── Basketball
│   │   └── v_shooting_01_01
│   └── Jumping
│       └── v_jumping_01_03

    The Sturctre Is Diffrent That Will Affect When We Reading Videos As Training Require Two
    Loops And Testing Require One Loop To Read Videos So We Create A Folder In Testing_set
    And Move All Videos To It
'''

!mkdir /content/Testing_set/test
!mv /content/Testing_set/*.mpg /content/Testing_set/test

In [ ]:
#@title Feature Extraction { form-width: "3%" }

def we_are_equal(videos_features, average_frames_count):
    """   
    Make Frames For Each Video Equal
  
    Parameters: 
    numpy array : All Videos Features 
    int         : Average Frames

    Returns: 
    numpy array :  Numpy Arrays Each Element Is For A Video Each Video Has The Same Frame Count
    """
 
    for video_index in range(len(videos_features)):
        # Set Single video [#Frames, #Features]
        temp_single_video = videos_features[video_index]
        temp_frames = temp_single_video.shape[0]

        if temp_frames > average_frames_count:
            temp_single_video = temp_single_video[0:average_frames_count, :]
            videos_features[video_index] = temp_single_video

        elif temp_frames < average_frames_count:
            temp = np.zeros(shape=(average_frames_count, temp_single_video.shape[1]))
            temp[0:temp_frames, :] = temp_single_video
            videos_features[video_index] = temp
    return videos_features

def get_videos_features():
    """   
    Get Features Of Each Frame For Each Video
  
    Parameters: 
    None 
  
    Returns: 
    List(numpy array) : A List Of Numpy Arrays Each Aarray's Sahpe [#Frames, Feature Of Each Frame]
    """

    print('Preprocessing videos')

    # This Model Responsible For Extracting Features For Each Frame
    # VGG
    spical_model = VGG16(include_top=False, weights='imagenet')
    spical_model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Inception
    # spical_model = inception_v3.InceptionV3(include_top=False, weights='imagenet')
    # spical_model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Global Variables
    image_size = 244
    all_video_features = []
    all_video_label = []
    all_videos_frames = []
    all_videos_id = []

    skip_millseconds = 500 # 1000 = 1 second
    # Loop Through Each Video
    if MASTER_FLAG :
        train_or_test_path = TRAINING_PATH_FOLDER
    else:
        train_or_test_path = TESTING_PATH_FOLDER
    for folder_name in tqdm(os.listdir(train_or_test_path)):
        for video_name in os.listdir(train_or_test_path + '/' + folder_name):
            video_path = train_or_test_path + '/' + folder_name + '/' + video_name

            # Multiply With skip_millseconds To Go To A Spacific Millisecond In A Vodeo
            # Also We Use This Variable As Frames Count For Each Video
            go_to_millsecond = 0 

            cap = cv2.VideoCapture(video_path)
            single_vidoe_features = []
            while True:

                # CAP_PROP_POS_MSEC Get Current Time In Millisecond Set Function Change This Flag
                cap.set(cv2.CAP_PROP_POS_MSEC, (go_to_millsecond*skip_millseconds))

                # Flag Will Be False When Reaching The End Of The Video
                flag, image = cap.read()
                
                if flag:
                    # Whithout This Condition  It Will Go In Infinite Loop
                    if(cap.get(cv2.CAP_PROP_POS_FRAMES) > cap.get(cv2.CAP_PROP_FRAME_COUNT)):
                        break

                    # Explanation For interpolation=cv2.INTER_AREA
                    # http://shorturl.at/bexXY
                    img = cv2.resize(image, (image_size, image_size), interpolation=cv2.INTER_AREA)

                    input = img_to_array(img)

                    # [224, 224] => [1, 224, 224] So We Can Feed It To The Model
                    input = np.expand_dims(input, axis=0)

                    # Explanation For preprocess_input
                    # https://stackoverflow.com/a/47556342/8236482
                    input = preprocess_input(input)

                    single_frame_features = spical_model.predict(input).ravel()
                    single_vidoe_features.append(single_frame_features)
                    go_to_millsecond = go_to_millsecond + 1
                else:
                    break
            all_videos_frames.append(go_to_millsecond)
            all_video_features.append(np.array(single_vidoe_features))
            # Get Video Label
            if MASTER_FLAG:
                for label in LABELS:
                    if label in video_name:
                        all_video_label.append(LABELS.index(label))
            else:
                all_videos_id.append(video_name)

    # We Will Use It In The Temporal Model
    hidden = np.array(all_video_features[0].shape[1])

    # Make All Videos Have The Same Count Of Frames
    mean = int(np.mean(np.asarray(all_videos_frames)))
    print('---------------------------------------------------')
    print('Avarge Frame:', mean)
    print('---------------------------------------------------')
    all_features = we_are_equal(all_video_features, mean)
    all_features = np.asarray(all_features)

    # Encode The Labels
    if MASTER_FLAG:
        all_video_label = np_utils.to_categorical(all_video_label, 5)
        all_video_label = np.asarray(all_video_label)
    else:
        all_videos_id = np.asarray(all_videos_id)

    # Saving 
    print('---------------------------------------------------')
    print('Saving Training Data')
    print('---------------------------------------------------')
    if MASTER_FLAG:
        np.save(TRAINING_NPY, all_features)
        np.save(TRAINING_LABEL_NPY, all_video_label)
    else:
        np.save(TESTING_NPY, all_features)
        np.save(TESTING_LABEL_NPY, all_videos_id)
    
    np.save(HIDDEN_NPY, hidden)

In [ ]:
#@title Model Requirements { form-width: "3%" }
 
def temporal_model(avg, lstm_par):
    """   
    Building temporal model layers
  
    Parameters: 
    int : LSTM Parametar
 
    Returns: 
Y   keras model :  The Model
    """
    temporal = Sequential()
    # memory 1
    # Explanition for LSTM parametars
    # https://www.youtube.com/watch?v=LZ-GS7LOyWs&t=334s
 
    temporal.add(
        Bidirectional(
            LSTM(units=256, return_sequences=True),
            input_shape=(avg, lstm_par)
        )
    )
    temporal.add(LSTM(200))
 
    
    
    # Normal Ending
    temporal.add(Dropout(0.55))
    temporal.add(Dense(5))
    temporal.add(Activation('softmax'))
    temporal.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return temporal
 
def batching(data, labels):
    """   
    Generate Batches
 
    Parameters: 
    numpy array : Features Of Each Video
    numpy array : Label Of Each Video
 
    Returns: 
    numpy array : Segments Of Data
    """
    batches_count = len(data) // 32
    while True:
        for index in range(0, batches_count):
            begin_batch = 32 * index 
            end_batch = 32 * (index + 1)
            # Explanition For Yield
            # shorturl.at/tVW37
            yield np.array(data[begin_batch:end_batch]), labels[begin_batch:end_batch]

In [ ]:
#@title Free Resources { form-width: "3%" }

# Trying To Free Colab GPU & Memory
cuda.select_device(0)
cuda.close()
os.kill(os.getpid(), 9)

In [ ]:
#@title Main { form-width: "3%" }

split_percentage = 0.2
start_epoch = 3
to_epoch = 10
print_red_on_yellow = lambda x: cprint(x, 'grey', 'on_yellow')

for epoch_index in range(start_epoch, to_epoch):
    print_red_on_yellow('---------------------------------------Epoch: {}---------------------------------------'.format(epoch_index))
    for flag_filter in range(2):
        if(flag_filter == 0):
            MASTER_FLAG = True
        else:
            MASTER_FLAG = False

        # True For Loading False For Saving
        LOAD_SAVE = True

        # Geting Or Prepare The Data
        if MASTER_FLAG:
            if LOAD_SAVE:
                print('---------------------------------------------------')
                print(colored('Loading Training Data And Train', 'green'))
                train = np.load(TRAINING_NPY, allow_pickle=True)
                train_label = np.load(TRAINING_LABEL_NPY, allow_pickle=True)

                # Just A Single Number 
                LSTM_parametar = int(np.load(HIDDEN_NPY, allow_pickle=True))
                avarge_frames = train[0].shape[0]

                # Get The Model
                final_model = temporal_model(avarge_frames, LSTM_parametar)

                # Split The Data
                Xtrain, Xtest, Ytrain, Ytest = train_test_split(train, train_label, test_size=split_percentage)

                # Batches Count
                b_train_count = len(Xtrain) / 32
                b_test_count = len(Xtest) / 32

                # Manully Generate Batches
                b_train = batching(Xtrain, Ytrain)
                b_test  = batching(Xtest, Ytest)

                # Finally Training The Model
                final_model.fit_generator(
                    generator=b_train, steps_per_epoch=b_train_count,
                    validation_data=b_test, validation_steps=b_test_count,
                    epochs=epoch_index, verbose = 0
                )
                final_model.save_weights(WEIGHTS_PATH)
            else:
                print('---------------------------------------------------')
                print(colored('Start Preprocessing Training Data And Saving', 'green'))
                get_videos_features()
        else:
            if LOAD_SAVE:
                print('---------------------------------------------------')
                print(colored('Loading Testing Data And Predict', 'green'))
                test = np.load(TESTING_NPY, allow_pickle=True)
                test_label = np.load(TESTING_LABEL_NPY, allow_pickle=True)
                LSTM_parametar = int(np.load(HIDDEN_NPY, allow_pickle=True))

                avarge_frames = test[0].shape[0]
                final_model = temporal_model(avarge_frames, LSTM_parametar)
                final_model.load_weights(WEIGHTS_PATH)

                # Videos ID's And Labels
                videos_ids_labels = dict()

                # Prepare Submition File
                open(CSV_PATH, "w").close()
                with open(CSV_PATH, 'w') as f:
                    f.write('Video,Label\n')
                for test_index in range(len(test)):
                    test_input = np.expand_dims(test[test_index], axis=0)
                    predicted_label = np.argmax(final_model.predict(test_input)[0])
                    with open(CSV_PATH, 'a') as f:
                        f.write('{},{}\n'.format(test_label[test_index], predicted_label))
                        videos_ids_labels.update( {test_label[test_index] : predicted_label} )

                # Importand This Code For Calculating The Accurcey
                true_positive = 0
                with open(CSV_SORTED_PATH) as csv_file:
                    csv_reader = csv.reader(csv_file, delimiter=',')
                    line_count = 0
                    for row in csv_reader:
                        if line_count == 0:
                            line_count += 1
                        else:
                            if(int(videos_ids_labels[row[0]]) == int(row[1])):
                                true_positive += 1

                print('---------------------------------------------------')
                # Some Colorful Stuff
                final_accurcy = (true_positive/127) * 100
                color = ''
                if final_accurcy >= 90:
                    color = 'green'
                elif final_accurcy < 90 and final_accurcy >= 80:
                    color = 'yellow'
                else:
                    color = 'red'

                print(colored('Final Accurecy: {}'.format(final_accurcy), color))
                print(colored('Submit CSV File', 'green'))
                print('---------------------------------------------------')
                
            else:
                print('---------------------------------------------------')
                print(colored('Start Preprocessing Testing Data And Saving', 'green'))
                get_videos_features()

In [ ]:
#@title Didn't know there is CSV Preprocessing { form-width: "3%" }
# For Only One Run The Csv Sorted File Will Be Provided

print('---------------------------------------------------')
print('Start CSV Preprocessing')
print('---------------------------------------------------')

videos_labels_sorted = []
sorted_labels = []

# Extracting Data From CSV File
with open(CSV_PATH) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            videos_labels_sorted.append(row)
            line_count += 1

# Sort The Data Into CSV File
for index in range(128):
    for sublist in videos_labels_sorted:
        exploded = re.split('. |\(|\)', sublist[0])
        if str(index) == exploded[2]:
            sorted_labels.append(sublist)

# Save The Data Into CSV File
open(CSV_SORTED_PATH, "w").close()
with open(CSV_SORTED_PATH, 'w') as f:
    f.write('Video,Label\n')
for video_info in sorted_labels:
    with open(CSV_SORTED_PATH, 'a') as f:
        f.write('{},{}\n'.format(video_info[0], video_info[1]))

print('---------------------------------------------------')
print('CSV Preprocessing End Successfully')
print(colored('NOW YOU NEED TO MANUALLY LABEL EACH VIDEO IN SORTED CSV', 'green'))
print('---------------------------------------------------')

In [ ]:
#@title End Of Project { form-width: "3%" }
%%html
<marquee style='width: 75%; color:black;'><h3> Best Accurcy : 92 % </h3></marquee>